In [9]:
!pip3 install -r requirements.txt

In [10]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [11]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are a culinary assistant helping with cooking and recipe questions. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically


@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [12]:
question = "What are the essential ingredients for making a perfect pasta sauce?"
ai_answer = langsmith_rag(question)
print(ai_answer)

The essential ingredients for making a perfect pasta sauce typically include tomatoes (fresh or canned), garlic, olive oil, salt, and fresh herbs like basil or oregano. You can also add onions, red pepper flakes, and a splash of wine for extra flavor. Adjust the ingredients based on your preferred sauce style, such as marinara or arrabbiata.


In [13]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "sklearn", "domain": "cooking"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [14]:
	
question = "How long should I cook chicken breast to ensure it's safe and tender?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Cook chicken breast to an internal temperature of 165°F (75°C) for safety. Typically, this takes about 20-30 minutes when baking at 375°F (190°C) or 6-8 minutes per side when grilling. Always use a meat thermometer to check for doneness.


In [15]:
question = "What temperature should I preheat my oven for baking chocolate chip cookies?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "baking_query"}})
print(ai_answer)

You should preheat your oven to 350°F (175°C) for baking chocolate chip cookies. This temperature allows the cookies to bake evenly and achieve a nice texture. Make sure to check for doneness a few minutes before the recommended baking time.
